In [1]:
#!pip install pandas openpyxl

In [1]:
import pandas as pd
df500_closingPrice = pd.read_excel(r"中证500股指期货收盘价(主力合约)2022-12-31.xlsx", engine='openpyxl')


In [2]:
df500_closingPrice.head()

,指标名称,中证500股指期货:收盘价(主力合约)
0,频率,日
1,单位,元/张
2,指标ID,M004349674
3,2022-12-30,5870.8
4,2022-12-29,5869.4


In [3]:
#delete first three rows 
df500_closingPrice = df500_closingPrice.iloc[3:]
#change feature name to "date" and "close"
df500_closingPrice.columns = ["date", "close"]
df500_closingPrice.head()

,date,close
3,2022-12-30,5870.8
4,2022-12-29,5869.4
5,2022-12-28,5882.4
6,2022-12-27,5897.6
7,2022-12-26,5860.6


In [4]:
df500_closingPrice['date']=pd.to_datetime(df500_closingPrice['date'])
df500_closingPrice.set_index("date", inplace=True)
m_groups=df500_closingPrice.groupby(pd.Grouper(freq='M'))
m_closingPrice=m_groups.mean()
m_closingPrice


,close
date,
2015-04-30,8104.218182
2015-05-31,9073.130000
2015-06-30,10376.019048
2015-07-31,7392.539130
2015-08-31,7353.590476
...,...
2022-08-31,6312.982609
2022-09-30,6030.066667
2022-10-31,5883.362500


In [5]:
#add index week1 week2 week3...
m_closingPrice['m'] = range(1, len(m_closingPrice) + 1)
m_closingPrice


,close,m
date,,
2015-04-30,8104.218182,1
2015-05-31,9073.130000,2
2015-06-30,10376.019048,3
2015-07-31,7392.539130,4
2015-08-31,7353.590476,5
...,...,...
2022-08-31,6312.982609,89
2022-09-30,6030.066667,90
2022-10-31,5883.362500,91


In [6]:
# make close column value natural log of close
import numpy as np
m_closingPrice['close'] = np.log(m_closingPrice['close'])
m_closingPrice

,close,m
date,,
2015-04-30,9.000140,1
2015-05-31,9.113073,2
2015-06-30,9.247253,3
2015-07-31,8.908227,4
2015-08-31,8.902944,5
...,...,...
2022-08-31,8.750364,89
2022-09-30,8.704513,90
2022-10-31,8.679884,91


In [7]:
#check missing value
m_closingPrice.isnull().sum()
#find out missing value
m_closingPrice[m_closingPrice.isnull().values==True]


,close,m
date,,


In [8]:
#drop missing value
m_closingPrice.dropna(inplace=True)


In [9]:
m_closingPrice.set_index("m", inplace=True)
m_closingPrice

,close
m,
1,9.000140
2,9.113073
3,9.247253
4,8.908227
5,8.902944
...,...
89,8.750364
90,8.704513
91,8.679884


In [10]:
#diff function
m_closingPrice['closingPrice'] = m_closingPrice['close'].diff(periods=1)
m_closingPrice

,close,closingPrice
m,,
1,9.000140,NaN
2,9.113073,0.112933
3,9.247253,0.134180
4,8.908227,-0.339026
5,8.902944,-0.005283
...,...,...
89,8.750364,-0.000160
90,8.704513,-0.045850
91,8.679884,-0.024630


In [11]:
#delete closing_price column
del m_closingPrice['close']
m_closingPrice

,closingPrice
m,
1,NaN
2,0.112933
3,0.134180
4,-0.339026
5,-0.005283
...,...
89,-0.000160
90,-0.045850
91,-0.024630


In [12]:
#to csv
m_closingPrice.to_csv(r'500_Future_return_monthly.csv', index = False, header=True)
